In [1]:
from ogdf_python import *

Tile_length = 200.0

cppinclude("ogdf/fileformats/GraphIO.h")
cppinclude("ogdf/basic/graph_generators/deterministic.h")
cppinclude("ogdf/fileformats/GraphIO.h")
cppinclude("ogdf/orthogonal/OrthoLayout.h")
cppinclude("ogdf/planarity/EmbedderMinDepthMaxFaceLayers.h")
cppinclude("ogdf/planarity/PlanarSubgraphFast.h")
cppinclude("ogdf/planarity/PlanarizationLayout.h")
cppinclude("ogdf/planarity/SubgraphPlanarizer.h")
cppinclude("ogdf/planarity/VariableEmbeddingInserter.h")


# cppinclude("ogdf/layered/SugiyamaLayout.h")

ogdf.EdgeArrow

G = ogdf.Graph()
ogdf.completeGraph(G, 64)
GA = ogdf.GraphAttributes(G, ogdf.GraphAttributes.all)
# define graph attributes
GA.directed=False

for n in G.nodes:
    GA.label[n] = "%s" % n.index()
    GA.width[n] = Tile_length
    GA.height[n] = Tile_length

for e in G.edges:
    GA.arrowType[e] =0

# SL = ogdf.PlanarizationLayout()
# SL.call(GA)
# GA

# Initialize layout components
pl = ogdf.PlanarizationLayout()
crossMin = ogdf.SubgraphPlanarizer()
ps = ogdf.PlanarSubgraphFast[int]()
ps.runs(100)
ves = ogdf.VariableEmbeddingInserter()
ves.removeReinsert(ogdf.RemoveReinsertType.All)

# Set up PlanarizationLayout components
crossMin.setSubgraph(ps)
crossMin.setInserter(ves)
pl.setCrossMin(crossMin)

# emb = ogdf.EmbedderMinDepthMaxFaceLayers()
# pl.setEmbedder(emb)

ol = ogdf.OrthoLayout()
ol.separation(10.0)
ol.cOverhang(0.4)

pl.setPlanarLayouter(ol)

pl.call(GA)
GA

: 

In [ ]:
pl.numberOfCrossings()

In [ ]:
ogdf.GraphIO.write(GA, "64_complete.gml")

In [ ]:
def calculate_edge_length(edge):
    length = 0
    points = edge['graphics']['Line']
    for i in range(len(points) - 1):
        x1, y1 = points[i]['x'], points[i]['y']
        x2, y2 = points[i + 1]['x'], points[i + 1]['y']
        length += abs(x2 - x1) + abs(y2 - y1)
    return length


def read_x_y(line):
    # e.g., line = '\t\t\t\tpoint [ x 230.0000000000 y 540.0000000000 ]\n'
    # Extract x and y values without using re
    start_index = line.find('x') + 1
    end_index = line.find('y')
    x_str = line[start_index:end_index].strip()
    y_str = line[end_index + 1 : -3].strip()  # -3 to exclude the closing bracket and newline
    # Convert to float
    x = float(x_str)
    y = float(y_str)
    return(x, y)


with open('64_complete.gml', 'r') as file:
    lines = file.readlines()

edges = []
current_edge = None

for line in lines:
    if 'edge' in line:
        current_edge = {'graphics': {'Line': []}}
    elif 'source' in line and current_edge is not None:
        current_edge['source'] = int(line.split()[1])
    elif 'target' in line and current_edge is not None:
        current_edge['target'] = int(line.split()[1])
    elif (']' in line) and (current_edge is not None) and (not ('point' in line)):
        edges.append(current_edge)
        current_edge = None
    elif 'point' in line and current_edge is not None:
        # Extract coordinates of points in the line segment
        coordinates = read_x_y(line)
        point = {'x': coordinates[0], 'y': coordinates[1]}
        current_edge['graphics']['Line'].append( point)

edge_lengths=[]
for edge in edges:
    length = calculate_edge_length(edge)
    edge_lengths.append(length)
    print(f"Edge from {edge['source']} to {edge['target']} has length: {length}")


In [ ]:
Tile_real_length = 30 #[mm]
ratio=Tile_real_length/Tile_length
for i in range(len(edge_lengths)):
    edge_lengths[i]*=ratio

In [ ]:
edge_lengths

In [ ]:
from statistics import mean
mean(edge_lengths)

In [ ]:
max(edge_lengths)